In [2]:
%pip install pandas matplotlib seaborn numpy scikit-learn plotly sympy


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

In [4]:
df = pd.read_csv('data.csv')
df.head(10)

,Amount,Type,Belonging,Date
0,"29,800",Expense,Food,10/12/2025
1,"40,600",Expense,Food,10/12/2025
2,"100,000",Income,Transfered,10/5/2025
3,"39,900",Expense,Food,10/5/2025
4,"34,000",Expense,Food,10/5/2025
5,"16,000",Expense,Giving,9/26/2025
6,"17,500",Expense,Food,9/25/2025
7,"100,000",Income,Transfered,9/25/2025
8,"9,999",Income,Transfered,9/19/2025
9,"10,000",Income,Transfered,9/16/2025


In [5]:
# Remove thousands separators and convert to numeric (coerce invalid values to NaN)
df['Amount'] = pd.to_numeric(df['Amount'].astype(str).str.replace(',', '', regex=False), errors='coerce')

In [6]:
# Know the mean amount
df['Amount'].mean()

np.float64(34843.13333333333)

The average amount of money I manage is 34,843 COP. In general

In [7]:
Amount = np.array(df['Amount'])
Type = np.array(df['Type'])

ShapeType = Type.shape
print(ShapeType)
print(type(ShapeType))

(15,)
<class 'tuple'>


Turn Tuple into integer

In [8]:
ShapeInteger = "".join(map(str, ShapeType))
ShapeInteger = int(ShapeInteger)
print(ShapeInteger) 

15


Turn Amount and Type into an Array

In [9]:
groupedMean = df['Amount'] = df.groupby('Type')['Amount'].mean()
print(groupedMean)

Type
Expense    27516.444444
Income     45833.166667
Name: Amount, dtype: float64


- The average Expense value is 27,516.4 COP
- The average Income value is 45,833.16 COP
<br><br>
I have more income in average.

# How much probability of Gaining X value? <br> How much probability of Losing X value?

## Gaining?

How much probability there's in the array, that the next value is an Income over a Expense?

In [10]:
dfI = df[df['Type'] == 'Income']
dfI = len(dfI)
print(int(dfI))

6


### Prob of Income

In [11]:
resultI = dfI/ShapeInteger
print(resultI)

0.4


### Prob of Expense

In [12]:
dfE = df[df['Type'] == 'Expense']
dfE = len(dfE)
print(int(dfE))

9


In [13]:
resultE = dfE/ShapeInteger
print(resultE)

0.6


### Calculation

In the next time is purchase or income? Use Markov chains

#### Theory of Markov Chains

Markov chains is an Stochastic process (Is a collection of random variables, that means that is like a list of values that are simply random over time). Its helpfull for finances and modelling random sequences.
<br>
<br>
To make this we're going to state:
- State I = Income
- State E = Expense
<br>
<br>
Markov chain consists of a matrix... The matrix holds the probabilities of moving from one state to another

##### Income

In [14]:
import pandas as pd
import numpy as np

TypeFiltered = df[['Type']].reset_index(drop=True)

TypeSequence = TypeFiltered['Type'].tolist()
print(TypeSequence)

['Expense', 'Expense', 'Income', 'Expense', 'Expense', 'Expense', 'Expense', 'Income', 'Income', 'Income', 'Income', 'Expense', 'Expense', 'Expense', 'Income']


In [15]:
df2 = pd.DataFrame(TypeSequence, columns=['Current_State'])
df2['Next_State'] = df2['Current_State'].shift(-1)
df2 = df2.dropna()
print(df2)

   Current_State Next_State
0        Expense    Expense
1        Expense     Income
2         Income    Expense
3        Expense    Expense
4        Expense    Expense
5        Expense    Expense
6        Expense     Income
7         Income     Income
8         Income     Income
9         Income     Income
10        Income    Expense
11       Expense    Expense
12       Expense    Expense
13       Expense     Income


##### Calulations

In [16]:
import numpy as np

transition_matrix_df2 = pd.crosstab(df2['Current_State'], df2['Next_State'], normalize='index')
transition_matrix = transition_matrix_df2.values

print(transition_matrix_df2)
print(transition_matrix)
print("================================")
P_II_final = transition_matrix_df2.loc['Income', 'Income']
print(P_II_final)

Next_State      Expense    Income
Current_State                    
Expense        0.666667  0.333333
Income         0.400000  0.600000
[[0.66666667 0.33333333]
 [0.4        0.6       ]]
0.6


### Probability of choosing an Expense?

In [17]:
P_EI_final = transition_matrix_df2.loc['Expense', 'Income']
print(f"P(Income | Expense): {P_EI_final:.4f}")

P(Income | Expense): 0.3333


Given that my financial event was an Income, there is a $33\%$ chance (or a 1/3 probability) that my next financial event will be an Expense (a purchase). Meaning that there's a 67% prob of getting an Income.